In [1]:
import altair as alt
from vega_datasets import data

cars = data.cars.url

alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon:Q',
    y='Horsepower:Q',
    color='Origin:N'
)

alt.Chart(...)

In [127]:
import pandas as pd
import altair as alt
df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2018/2018-11-13/malaria_deaths.csv')
#df.head()
df.columns = ['name', 'ISO3166-1-Alpha-3','Year','Death_Rate']
#df['Year'] = pd.to_datetime(df['Year'], format='%Y')
df.head(20)

,name,ISO3166-1-Alpha-3,Year,Death_Rate
0,Afghanistan,AFG,1990,6.802930
1,Afghanistan,AFG,1991,6.973494
2,Afghanistan,AFG,1992,6.989882
3,Afghanistan,AFG,1993,7.088983
4,Afghanistan,AFG,1994,7.392472
5,Afghanistan,AFG,1995,7.405511
6,Afghanistan,AFG,1996,7.626773
7,Afghanistan,AFG,1997,7.592172
8,Afghanistan,AFG,1998,7.796014
9,Afghanistan,AFG,1999,7.800592


In [128]:
country_info = pd.read_csv("https://raw.githubusercontent.com/datasets/country-codes/master/data/country-codes.csv",dtype = 'str')

In [162]:
df_new = df[df['ISO3166-1-Alpha-3'].isin(country_info['ISO3166-1-Alpha-3'])]
df_new.head()

,name,ISO3166-1-Alpha-3,Year,Death_Rate
0,Afghanistan,AFG,1990,6.802930
1,Afghanistan,AFG,1991,6.973494
2,Afghanistan,AFG,1992,6.989882
3,Afghanistan,AFG,1993,7.088983
4,Afghanistan,AFG,1994,7.392472


In [163]:
df_final = pd.merge(df_new, country_info , on = 'ISO3166-1-Alpha-3', how = 'left')

In [164]:
df_final = df_final[['Year','Death_Rate','ISO3166-1-numeric']]
df_final.head()

,Year,Death_Rate,ISO3166-1-numeric
0,1990,6.802930,004
1,1991,6.973494,004
2,1992,6.989882,004
3,1993,7.088983,004
4,1994,7.392472,004


In [153]:
alt.data_transformers.disable_max_rows()
countries = alt.topo_feature(data.world_110m.url, 'countries')
df_final['Year'] = df_final['Year'].astype(str)
df_final = df_final.pivot_table(index='ISO3166-1-numeric', columns='Year', values='Death_Rate').reset_index()
columns = [str(year) for year in range(1990, 2016)]
slider = alt.binding_range(
    step=1,
    min=1990, 
    max=2016
)

select_date = alt.selection_single(
    name="slider", 
    fields=['Year'],
    bind=slider, 
)
'''
alt.Chart(countries).mark_geoshape()\
    .encode(color='Death_Rate:Q')\
    .add_selection(select_date)\
    .transform_filter(select_date)\
    .transform_lookup(
        lookup='id',
        from_=alt.LookupData(df_final, key='ISO3166-1-numeric', fields=["Death_Rate"])
    )\
    .project('equirectangular')\
    .properties(
        width=500,
        height=300,
        title='Title'
    )
'''
alt.Chart(countries).mark_geoshape(
    stroke='black',
    strokeWidth=0.05
).project(
    type='equirectangular'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df_final, 'ISO3166-1-numeric', columns)
).transform_fold(
    columns, as_ = ['Year', 'Death_Rate']
).transform_calculate(
    Year='parseInt(datum.Year)',
    #Death_Rate='isValid(datum.Death_Rate) ? datum.Death_Rate: -1'  
).encode(
    color = alt.condition(
        ' Death_Rate> 0',
        alt.Color('Death_Rate:Q', scale=alt.Scale(scheme='blues')),
        alt.value('#dbe9f6')
    )).add_selection(
    select_date
).properties(
    width=700,
    height=400
).transform_filter(
    select_date
)


alt.Chart(...)

In [161]:
alt.Chart(countries).mark_geoshape(
    stroke='black',
    strokeWidth=0.05
).project(
    type='equirectangular'
).transform_lookup(
    lookup='id',
    from_ =alt.LookupData(df_final, 'ISO3166-1-numeric', columns)
).transform_fold(
    columns, as_=['Key', 'Value']
).transform_calculate(
    Key='parseInt(datum.Year)',
    #Death_Rate='isValid(datum.Death_Rate) ? datum.Death_Rate: -1'  
).encode(
    color = alt.condition(
        ' Value> 0',
        alt.Color('Value:Q', scale=alt.Scale(scheme='blues')),
        alt.value('#dbe9f6')
    )).add_selection(
    select_date
).properties(
    width=700,
    height=400
).transform_filter(
    select_date
)


alt.Chart(...)

In [170]:
from vega_datasets import data
alt.data_transformers.disable_max_rows()
countries = alt.topo_feature(data.world_110m.url, 'countries')
slider = alt.binding_range(
    step=1,
    min=1990, 
    max=2016
)

select_date = alt.selection_single(
    name="slider", 
    fields=['Year'],
    bind=slider, 
)

alt.Chart(df_final).mark_geoshape()\
    .encode(color='Death_Rate:Q')\
    .add_selection(select_date)\
    .transform_filter(select_date)\
    .transform_lookup(
        lookup='ISO3166-1-numeric',
        from_=alt.LookupData(countries, key='id', fields=["type", "properties", "geometry"])
    )\
    .project('equirectangular')\
    .properties(
        width=500,
        height=300,
        title='Title'
    )


alt.Chart(...)

In [154]:
import altair as alt
import pandas as pd
from vega_datasets import data

us_counties = alt.topo_feature(data.us_10m.url, 'counties')
fdf = pd.read_csv('https://raw.githubusercontent.com/sdasara95/Opioid-Crisis/master/sample_data.csv')
fdf['year'] = fdf['year'].astype(str)
fdf = fdf.pivot(index='fips', columns='year', values='Pill_per_pop').reset_index()
columns = [str(year) for year in range(2006, 2013)]

slider = alt.binding_range(min=2006, max=2012, step=1)
select_year = alt.selection_single(name="year", fields=['year'],
                                   bind=slider, init={'year': 2006})

alt.Chart(us_counties).mark_geoshape(
    stroke='black',
    strokeWidth=0.05
).project(
    type='albersUsa'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(fdf, 'fips', columns)
).transform_fold(
    columns, as_=['year', 'Pill_per_pop']
).transform_calculate(
    year='parseInt(datum.year)',
    Pill_per_pop='isValid(datum.Pill_per_pop) ? datum.Pill_per_pop : -1'  
).encode(
    color = alt.condition(
        'datum.Pill_per_pop > 0',
        alt.Color('Pill_per_pop:Q', scale=alt.Scale(scheme='blues')),
        alt.value('#dbe9f6')
    )).add_selection(
    select_year
).properties(
    width=700,
    height=400
).transform_filter(
    select_year
)

alt.Chart(...)

In [159]:
df_final.head()

Year,ISO3166-1-numeric,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,004,6.802930,6.973494,6.989882,7.088983,7.392472,7.405511,7.626773,7.592172,7.796014,...,8.266378,8.077900,7.817882,7.271249,6.826629,6.611425,7.144628,7.137835,7.192283,7.139429
1,008,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,012,0.027130,0.027024,0.026476,0.026566,0.026699,0.026151,0.026465,0.026657,0.027148,...,0.024676,0.023874,0.023513,0.022629,0.022393,0.022439,0.021616,0.021732,0.021629,0.021669
3,016,0.325525,0.327410,0.322589,0.311008,0.304775,0.300823,0.297561,0.293052,0.282703,...,0.219236,0.212173,0.212929,0.212127,0.211896,0.209409,0.208585,0.208029,0.207377,0.206802
4,020,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
import altair as alt
from vega_datasets import data

counties = alt.topo_feature(data.us_10m.url, 'counties')
source = data.unemployment.url

alt.Chart(counties).mark_geoshape().encode(
    color='rate:Q'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(source, 'id', ['rate'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

alt.Chart(...)

In [24]:
print(source)

https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/unemployment.tsv


In [53]:
source = alt.topo_feature(data.world_110m.url, 'countries')
source

UrlData({
  format: TopoDataFormat({
    feature: 'countries',
    type: 'topojson'
  }),
  url: 'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json'
})

In [52]:
counties

UrlData({
  format: TopoDataFormat({
    feature: 'counties',
    type: 'topojson'
  }),
  url: 'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json'
})

In [157]:
import pandas as pd
import numpy as np
import altair as alt

data = { 'ColA': {('A', 'A-1'): 'w',
                 ('A', 'A-2'): 'w',
                 ('A', 'A-3'): 'w',
                 ('B', 'B-1'): 'q',
                 ('B', 'B-2'): 'q',
                 ('B', 'B-3'): 'r',
                 ('C', 'C-1'): 'w',
                 ('C', 'C-2'): 'q',
                 ('C', 'C-3'): 'q',
                 ('C', 'C-4'): 'r'},
        'ColB': {('A', 'A-1'): 'r',
                 ('A', 'A-2'): 'w',
                 ('A', 'A-3'): 'w',
                 ('B', 'B-1'): 'q',
                 ('B', 'B-2'): 'q',
                 ('B', 'B-3'): 'e',
                 ('C', 'C-1'): 'e',
                 ('C', 'C-2'): 'q',
                 ('C', 'C-3'): 'r',
                 ('C', 'C-4'): 'w'} 
        }
                 
df1 = pd.DataFrame(data).reset_index( drop = True )



In [158]:
df1

,ColA,ColB
0,w,r
1,w,w
2,w,w
3,q,q
4,q,q
5,r,e
6,w,e
7,q,q
8,q,r
9,r,w
